In [1]:
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, classification_report, plot_confusion_matrix
from keras.models import load_model

from utils import split_sequences

Using TensorFlow backend.


In [17]:
with open('configs.json', 'r') as file:
    configs = json.load(file)
 
MODEL_CHECKPOINT_PATH = configs["MODEL_CHECKPOINT_PATH"]
lookback_window = configs['LOOKBACK_WINDOW']
lookforward_window = configs['LOOKFORWARD_WINDOW']

df = pd.read_csv('dataset.csv')

df['readiness_group'] = pd.cut(df['readiness'], 3, labels=[0, 1, 2])

X, y = split_sequences(
    df,
    lookback_window,
    lookforward_window,
    y_variable_name='readiness_group',
    exclude_columns=['pid', 'date', 'readiness', 'readiness_group']
)

nsamples, nx, ny = X.shape
X = X.reshape((nsamples, nx * ny))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=configs["RANDOM_STATE"], shuffle=True)

# MinMaxScaler
minmax_scaler = MinMaxScaler()
X_train = minmax_scaler.fit_transform(X_train)
X_test = minmax_scaler.transform(X_test)

nsamples = X_train.shape[0]
X_train = X_train.reshape((nsamples, nx, ny))

nsamples = X_test.shape[0]
X_test = X_test.reshape((nsamples, nx, ny))

classifier = load_model(f'{MODEL_CHECKPOINT_PATH}/readiness_cnn_classifier_keras_6_1_10_3_64.hdf5')

y_train_pred = np.argmax(classifier.predict(X_train), axis=-1)
y_test_pred = np.argmax(classifier.predict(X_test), axis=-1)

print('On training data')
print('F1 weighted: ', f1_score(y_train, y_train_pred, average='weighted'))

print('On test data')
print('F1 weighted: ', f1_score(y_test, y_test_pred, average='weighted'))

On training data
F1 weighted:  0.7065247232430838
On test data
F1 weighted:  0.6801459817927591


In [18]:
print("Classification report for classifier %s:\n%s\n" % ('', classification_report(y_test, y_test_pred)))

Classification report for classifier :
              precision    recall  f1-score   support

           0       0.52      0.39      0.45        61
           1       0.77      0.80      0.79       215
           2       0.50      0.55      0.52        55

    accuracy                           0.69       331
   macro avg       0.60      0.58      0.59       331
weighted avg       0.68      0.69      0.68       331


